## Analyzing dome monitor with a labJack T7
Uses 2 analog input (AINs) to read the data.

Craig Lage - Jul 5, 2023

In [ ]:
import sys, os, re
import time, datetime
import numpy as np
import pandas as pd
import pickle as pkl
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient

In [ ]:
dir = "/scratch/labJackData/"

files = os.listdir(dir)
counter = 0
for file in files:
    split_file = re.split('_|T|\Z.', file)
    if split_file[0] == "Gill":
        if (split_file[1] == '20231112') or \
            (split_file[1] == '20231111' and int(list(split_file[2])[0]) > 1):
            filename = dir+file

            pkl_file = open(filename, 'rb')
            df = pkl.load(pkl_file)
            pkl_file.close()
            print(filename, len(df), df.columns)
            if counter == 0:
                merged_df = df
            else:
                merged_df = pd.concat([merged_df, df])
            #if counter == 1:
            #    break
            counter += 1


In [ ]:
print(len(df), len(merged_df))

In [ ]:
client = EfdClient('summit_efd')
start = Time("2023-11-11T20:38:00Z", scale='utc')
end = Time("2023-11-12T20:29:00Z", scale='utc')
campbellVels = await client.select_time_series('lsst.sal.ESS.airTurbulence', \
                                            ['*'],  start, end)
temps = await client.select_time_series('lsst.sal.ESS.temperature', \
                                            ['temperatureItem2'],  start, end)

In [ ]:
t0 = Time("2023-11-12T02:00:00Z", scale='utc').isot
t1 = Time("2023-11-12T09:00:00Z", scale='utc').isot
t2 = Time("2023-11-12T12:00:00Z", scale='utc').isot

sub_df = merged_df.loc[t0:t1]
sub_campbell =campbellVels.loc[t0:t1]
gill_sigma_vx = np.std(sub_df['vx_ms'].values)
gill_sigma_vy = np.std(sub_df['vy_ms'].values)
gill_sigma_vz = np.std(sub_df['vz_ms'].values)
campbell_sigma_vx = np.std(sub_campbell['speed0'].values)
campbell_sigma_vy = np.std(sub_campbell['speed1'].values)
campbell_sigma_vz = np.std(sub_campbell['speed2'].values)

In [ ]:
print(len(campbellVels), len(df), len(merged_df))

In [ ]:
fig, axs = plt.subplots(3,1, figsize=(6,12))
plt.subplots_adjust(hspace=0.5)

nb_row = 6000 # rows to average
plot0 = merged_df['vx_ms'].rolling(nb_row).mean()[nb_row::nb_row].plot(ax=axs[0], label="Gill_vx", color='red')
merged_df['vy_ms'].rolling(nb_row).mean()[nb_row::nb_row].plot(ax=axs[0], label="Gill_vy", color='blue')
merged_df['vz_ms'].rolling(nb_row).mean()[nb_row::nb_row].plot(ax=axs[0], label="Gill_vz", color='green')
plot0.axvline(t0, ls='--', color='k')
plot0.axvline(t1, ls='--', color='k')
axs[0].set_ylim(-0.2, 0.2)
axs[0].set_ylabel("Wind speed(m/s)")
axs[0].set_title("Gill wind velocities", fontsize=16)
axs[0].legend(loc='upper left')
axs[0].text(t2, -0.08, f"sigmaVx={gill_sigma_vx:.3f}")
axs[0].text(t2, -0.12, f"sigmaVy={gill_sigma_vy:.3f}")
axs[0].text(t2, -0.16, f"sigmaVz={gill_sigma_vz:.3f}")

plot1 = campbellVels["speed0"].plot(ax=axs[1], label="Campbell_vx", color='red')
campbellVels["speed1"].plot(ax=axs[1], label="Campbell_vy", color='blue')
campbellVels["speed2"].plot(ax=axs[1], label="Campbell_vz", color='green')
plot1.axvline(t0, ls='--', color='k')
plot1.axvline(t1, ls='--', color='k')
axs[1].set_ylim(-0.2, 0.2)
axs[1].set_ylabel("Wind speed(m/s)")
axs[1].set_title("Campbell wind velocities", fontsize=16)
axs[1].legend(loc='upper left')
axs[1].text(t2, -0.08, f"sigmaVx={campbell_sigma_vx:.3f}")
axs[1].text(t2, -0.12, f"sigmaVy={campbell_sigma_vy:.3f}")
axs[1].text(t2, -0.16, f"sigmaVz={campbell_sigma_vz:.3f}")


merged_df['tsonic'].rolling(nb_row).mean()[nb_row::nb_row].plot(ax=axs[2], label="Gill_tsonic", color='red')
campbellVels["sonicTemperature"].plot(ax=axs[2], label="Campbell_tsonic", color='blue')
temps["temperatureItem2"].rolling(nb_row).mean()[nb_row::nb_row].plot(ax=axs[2], label="Air temperature", color='green')
axs[2].set_ylim(5.0, 20.0)
axs[2].set_ylabel("Temperature (C)")
axs[2].set_title("Sonic Temperature", fontsize=16)
axs[2].legend(loc='lower left')


plt.savefig(f"/home/craiglagegit/DATA/Gill_vs_Campbell_11Nov23.png")